# Optimize the dataset using crocoddyl

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import roslib
import tf
from memmo_utils import *
from database import *
from croc_lib import *
import os
from geometry_msgs.msg import Quaternion, Pose, Point, Vector3
from std_msgs.msg import Header, ColorRGBA
import rospy
import os
from mlp.utils.status import Status
import mlp.utils.wholebody_result as wb_res
import matplotlib.pyplot as plt
import pickle
from IPython.display import clear_output

from transforms3d import axangles
from transforms3d import affines, quaternions

from pykdl_utils.kdl_parser import kdl_tree_from_urdf_model
from pykdl_utils.kdl_kinematics import KDLKinematics

import crocoddyl.utils as utils
from regression import *

np.set_printoptions(precision=4, suppress=True)

%load_ext autoreload
%autoreload 2


FILE_NAME = 'learn_onestep'
%matplotlib inline


### Start Instruction

### Load Robot using URDF 

In [ ]:
import roslib; roslib.load_manifest('urdfdom_py')
from urdf_parser_py.urdf import URDF
robot = URDF.from_parameter_server()
clear_output()

tree = kdl_tree_from_urdf_model(robot)

left_foot = KDLKinematics(robot, 'base_link', 'left_sole_link')
right_foot = KDLKinematics(robot, 'base_link', 'right_sole_link')

q = left_foot.random_joint_angles()
pose = left_foot.forward(q) # forward kinematics (returns homogeneous 4x4 numpy.mat)
q_ik = left_foot.inverse(pose, q+0.3) # inverse kinematics

### Create Visualization

In [ ]:
viz = Visual()
viz.rate = rospy.Rate(1000)

foot_marker = RvizMarker('foot_marker', 5, Marker.CUBE, 4)
colors = [ColorRGBA(0.0, 1.0, 0.0, 0.3), ColorRGBA(0.0, 0.0, 1.0, 0.3)] + [ColorRGBA(1.0, 0.0, 0.0, 0.9)]*2
foot_marker.set_color(colors)

### Loading Files

In [ ]:
FILE_CAT = 'talos_moveEffector_flat'
#FILE_CAT = 'talos_circle'
#FILE_CAT = 'talos_circle_oriented'
ROOT_PROCESSED = '/media/teguh/Data/MEMMO Dataset/processed_data/' + FILE_CAT + '/'

f = open(ROOT_PROCESSED + '/data_left_right.pkl', 'rb')
data = pickle.load(f)
f.close()

trajs = dict()
x_inputs = dict()
foot_poses = dict()

In [ ]:
data_phases_ori = data['left']['data_phases_set'][0]

goal_type = 'contact'#'root'#

keys = ['left','right']
for key in keys:
    trajs[key] = data[key]['trajs'][0:1000]
    if goal_type == 'contact':
        x_inputs[key] = data[key]['x_inputs'][0:1000]
    elif goal_type == 'root':
        x_inputs[key] = data[key]['x_inputs_root'][0:1000]
    foot_poses[key] = data[key]['foot_poses'][0:1000]

# Warmstart crocoddyl

In [ ]:
def calc_foot_T(x, move = 'left'):
    #calculate the footstep transformation matrix based on the input x
    #x is defined as: [left_foot, right_foot, 'foot_to_move']
    x = x.reshape(-1,3)
    Ts = []
    for x_i in x:
        T = PosetoMat(x_i)
        Ts.append(T)
    
    if move == 'left':
        T_lefts = [Ts[0], None, Ts[2]]
        T_rights = [Ts[1]]*3
    else:
        T_rights = [Ts[1], None, Ts[2]]
        T_lefts = [Ts[0]]*3
        
    return T_lefts, T_rights

## 1. Try one problem

#### Create a problem

In [ ]:
if np.random.rand() < 0.5:
    key = 'left'
else:
    key = 'right'

index = np.random.randint(len(x_inputs[key]))
x = x_inputs[key][index]

T_lefts, T_rights = calc_foot_T(x,move=key)
foot_marker.publish(x.reshape(-1,3))

traj = trajs[key][index]

#visualize
viz.set_rate(1000)
viz.vis_traj(traj)

#### Create phases

In [ ]:
data_phases = subsample_phases(data_phases_ori.copy(),40)

In [ ]:
delta_t = 0.04

#### Subsample the trajectories 

In [ ]:
T = traj.shape[0]
intervals = np.concatenate([np.arange(0,T,40),[T-1]])
T_new = len(intervals)
traj = traj[intervals]
vel_traj = np.zeros((traj.shape[0], 38))
u_traj = np.zeros((traj.shape[0], 32))

#for visualization
viz.set_rate(25)
raw_traj = traj.copy()

In [ ]:
viz.vis_traj(raw_traj)

#### Store in Crocoddyl format 

In [ ]:
phases = store_for_crocoddyl(traj, vel_traj, u_traj, T_lefts, T_rights, data_phases, delta_t)
pickle.dump(phases,open('data_teguh_3.txt','wb'))

### Run Crocoddyl 

In [ ]:
from memmo_utils import *
from crocoddyl import loadTalos
from croc_lib import *

In [ ]:
ROBOT = loadTalos('/opt/openrobots/share/example-robot-data/robots/')
problem, xs, us, ts = define_croc_problem(ROBOT,'data_teguh_3.txt', num_phases=3, is_warmstart=True)
clear_output()

solver = solve_problem(ROBOT,problem,xs, us,maxiter = 50, STOP_THRESHOLD=1e-03, recalc_u=True, TYPE_OF_SOLVER='FDDP')
traj = np.array(solver.xs)[:,0:39]

viz.vis_traj(traj)

#### A quick check

## Optimize the whole dataset 

#### Convert to crocoddyl problem format 

In [ ]:
to_load = False
if to_load:
    FILE_CAT = 'talos_moveEffector_flat'
    ROOT_CROCS = '/media/teguh/Data/MEMMO Dataset/crocs_data/' + FILE_CAT + '/'

    f = open(ROOT_CROCS + 'data_left_right_5th.pkl', 'rb')
    crocs_data = pickle.load(f)
    print 'Data has been loaded!'
    f.close()
else:
    crocs_data = dict()
    crocs_data['left'] = dict()
    crocs_data['right'] = dict()
    for key in crocs_data.keys():
        crocs_data[key]['foot_poses'] = []
        crocs_data[key]['trajs'] = []
        crocs_data[key]['x_inputs'] = []
        crocs_data[key]['vel_trajs'] = []        
        crocs_data[key]['u_trajs'] = []
        crocs_data[key]['data_phases_set'] = []
        crocs_data[key]['costs'] = []
        crocs_data[key]['iters'] = []
        

In [ ]:
ROBOT = loadTalos('/opt/openrobots/share/example-robot-data')
viz.set_rate(25)

#### Only create the croc files 

In [ ]:
file_name = 'data.txt'
FILE_CAT = 'talos_moveEffector_flat'
ROOT_CROCS = '/media/teguh/Data/MEMMO Dataset/crocs_data/' + FILE_CAT + '/'

tic = time.time()
for key in crocs_data.keys():
    for i in range(len(trajs[key])):
        traj = trajs[key][i]
        x_input = x_inputs[key][i]
        q_init = traj[0].copy()
        v_init = np.zeros(38)
        file_name = ROOT_CROCS + '/crocs_files/data_' + key + '_' + str(i) + '.txt'
        traj_mod,data_phases = convert_to_croc_format(x_input,traj,key,q_init = q_init, v_init = v_init, data_phases = data_phases_ori, filename=file_name)

#### Examine the croc files 

In [ ]:
for key in crocs_data.keys():
    for i in range(10):#len(trajs[key])):
        file_name = ROOT_CROCS + '/crocs_files/data_' + key + '_' + str(i) + '.txt'
        f = open(file_name,'rb')
        phases = pickle.load(f)
        examine_phases(phases,left_foot,right_foot,viz, foot_marker,False)
        raw_input()
        clear_output()
        f.close()

#### Optimize using crocoddyl

In [ ]:
file_name = 'data.txt'
tic = time.time()
for key in crocs_data.keys():
#for key in ['left']:    
    for i in range(850,1000):#len(trajs[key])):
        traj = trajs[key][i]
        x_input = x_inputs[key][i]
        q_init = traj[0].copy()
        v_init = np.zeros(38)
        traj_mod,data_phases = convert_to_croc_format(x_input,traj,key,q_init = q_init, v_init = v_init, data_phases = data_phases_ori, filename=file_name)
        problem, xs, us, ts = define_croc_problem(ROBOT,file_name, num_phases=3, is_warmstart=True)
        clear_output()
        toc = time.time()
        print 'Time so far: ' + str(toc-tic)
        
        print 'Iteration '  + key + '_' + str(i)
        solver = solve_problem(ROBOT,problem,xs, us,maxiter = 50, STOP_THRESHOLD=1e-05, recalc_u=True, TYPE_OF_SOLVER='FDDP')
        traj = np.array(solver.xs)[:,0:39]
        vel_traj = np.array(solver.xs)[:,39:]
        crocs_data[key]['trajs'].append(traj)
        crocs_data[key]['vel_trajs'].append(vel_traj)
        crocs_data[key]['u_trajs'].append(solver.us)
        crocs_data[key]['x_inputs'].append(x_input)
        crocs_data[key]['foot_poses'].append(foot_poses[key][i])
        crocs_data[key]['data_phases_set'].append(data_phases)
        crocs_data[key]['costs'].append(solver.cost)
        crocs_data[key]['iters'].append(solver.iter)
        
        #foot_marker.publish(foot_poses[key][i])
        #viz.vis_traj(np.array(solver.xs)[:,0:39])
data['information'] = 'One step movement, created from ' + FILE_CAT + '. The trajectories are transformed s.t. the \
root positions start from zero. The data are separated into right and left. This dataset is optimized by Crocoddyl.'

#### Save Data 

In [ ]:
FILE_CAT = 'talos_moveEffector_flat'
ROOT_CROCS = '/media/teguh/Data/MEMMO Dataset/crocs_data/' + FILE_CAT + '/'

f = open(ROOT_CROCS + 'data_left_right.pkl', 'wb')
pickle.dump(crocs_data,f)
print 'Data has been saved!'
f.close()

In [ ]:
index = np.random.randint(len(crocs_data[key]['x_inputs']))
key = 'left'
foot_marker.publish(crocs_data[key]['x_inputs'][index].reshape(-1,3))
viz.vis_traj(crocs_data[key]['trajs'][index])

#### For multi-step movements 

In [ ]:
FILE_CAT = 'talos_circle_oriented'
ROOT_CROCS = '/media/teguh/Data/MEMMO Dataset/crocs_data/' + FILE_CAT + '/'

delta_t = 0.04
trajs_test = data['trajs']
T_lefts_set = data['T_lefts_set']
T_rights_set = data['T_rights_set']
data_phases_set = data['data_phases_set']

In [ ]:
file_name = 'data.txt'
FILE_CAT = 'talos_moveEffector_flat'
ROOT_CROCS = '/media/teguh/Data/MEMMO Dataset/crocs_data/' + FILE_CAT + '/'

tic = time.time()
for key in crocs_data.keys():
    for i in range(len(trajs[key])):
        traj = trajs[key][i]
        x_input = x_inputs[key][i]
        q_init = traj[0].copy()
        v_init = np.zeros(38)
        file_name = ROOT_CROCS + '/crocs_files/data_' + key + '_' + str(i) + '.txt'
        traj_mod,data_phases = convert_to_croc_format(x_input,traj,key,q_init = q_init, v_init = v_init, data_phases = data_phases_ori, filename=file_name)

In [ ]:
for index in range(len(trajs_test)):
    traj = trajs_test[index]
    T_lefts = T_lefts_set[index]
    T_rights = T_rights_set[index]
    data_phases_full = data_phases_set[index]
    T_orig = traj.shape[0]

    #subsample
    data_phases = subsample_phases(data_phases_full,sub_fac=40)

    intervals = np.concatenate([np.arange(0,T_orig, 40), [T_orig-1]])
    T = data_phases[-1][-1]+1
    traj = subsample(traj,T)
    vel_traj = np.zeros((traj.shape[0], 38))
    u_traj = np.zeros((traj.shape[0], 32))

    #### Set the height of the foot to zero 
    for T in T_lefts:
        if T is not None: T[2,3] = 0.
    for T in T_rights:
        if T is not None: T[2,3] = 0.
    phases = store_for_crocoddyl(traj, vel_traj, u_traj, T_lefts, T_rights, data_phases, delta_t)
    file_name = ROOT_CROCS + '/crocs_files/data_' + str(index) + '.txt'
    pickle.dump(phases,open(file_name,'wb'))

#### Examine the croc files 

In [ ]:
for i in range(10):#len(trajs[key])):
    file_name = ROOT_CROCS + '/crocs_files/data_'  + str(i) + '.txt'
    f = open(file_name,'rb')
    phases = pickle.load(f)
    examine_phases(phases,left_foot,right_foot,viz, foot_marker,False)
    raw_input()
    clear_output()
    f.close()